In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")
import numpy as np
import scanpy.api as sc
from anndata import read_h5ad
from anndata import AnnData
import scipy as sp
import scipy.stats
from gprofiler import GProfiler
import pickle
# Other specific functions 
from itertools import product
from statsmodels.stats.multitest import multipletests
import util
# R related packages 
import rpy2.rinterface_lib.callbacks
import logging
from rpy2.robjects import pandas2ri
import anndata2ri

/home/martin/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)
# Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython
# autoreload
%load_ext autoreload
%autoreload 2
# logging
sc.logging.print_versions()

scanpy==1.4.3 anndata==0.6.20 umap==0.3.8 numpy==1.16.4 scipy==1.2.1 pandas==0.25.0 scikit-learn==0.21.1 statsmodels==0.9.0 python-igraph==0.7.1 louvain==0.6.1 


In [3]:
%%R
library(MAST)

## Load data

In [4]:
# Data path
data_path = '/data3/martin/tms_gene_data'
output_folder = data_path + '/DE_result'

In [5]:
# Load the data 
adata_combine = util.load_normalized_data(data_path)

In [6]:
temp_facs = adata_combine[adata_combine.obs['b_method']=='facs',]
temp_droplet = adata_combine[adata_combine.obs['b_method']=='droplet',]

## Generate a list of tissues for DE testing

In [7]:
tissue_list = list(set(temp_droplet.obs['tissue']))
min_cell_number = 1
analysis_list = []
analysis_info = {}
# for cell_type in cell_type_list:
for tissue in tissue_list:
    analyte = tissue
    ind_select = (temp_droplet.obs['tissue'] == tissue)
    n_young = (temp_droplet.obs['age'][ind_select].isin(['1m', '3m'])).sum()
    n_old = (temp_droplet.obs['age'][ind_select].isin(['18m', '21m',
                                                   '24m', '30m'])).sum()
    analysis_info[analyte] = {}
    analysis_info[analyte]['n_young'] = n_young
    analysis_info[analyte]['n_old'] = n_old
    if (n_young>min_cell_number) & (n_old>min_cell_number):
        print('%s, n_young=%d, n_old=%d'%(analyte, n_young, n_old))
        analysis_list.append(analyte)

Tongue, n_young=12044, n_old=8613
Heart_and_Aorta, n_young=1362, n_old=6554
Lung, n_young=6541, n_old=21216
Spleen, n_young=7844, n_old=21478
Liver, n_young=3234, n_old=3246
Bladder, n_young=3450, n_old=5367
Limb_Muscle, n_young=8210, n_old=16759
Thymus, n_young=1145, n_old=6425
Kidney, n_young=4317, n_old=14784
Marrow, n_young=6842, n_old=35099
Mammary_Gland, n_young=4343, n_old=7049


### DE using R package MAST 

In [8]:
## DE testing
gene_name_list = np.array(temp_droplet.var_names)
DE_result_MAST = {}
for i_analyte,analyte in enumerate(analysis_list):
    print(analyte, '%d/%d'%(i_analyte, len(analysis_list)))
    tissue = analyte
    ind_select = (temp_droplet.obs['tissue'] == tissue)
    adata_temp = temp_droplet[ind_select,]
    # reformatting
    adata_temp.X = np.array(adata_temp.X.todense())
    adata_temp.obs['condition'] = [int(x[:-1]) for x in adata_temp.obs['age']] 
    adata_temp.obs = adata_temp.obs[['condition', 'sex']]
    if len(set(adata_temp.obs['sex'])) <2:
        covariate = ''
    else:
        covariate = '+sex'
#     # toy example
#     covariate = ''
#     np.random.seed(0)
#     ind_select = np.random.permutation(adata_temp.shape[0])[0:100]
#     ind_select = np.sort(ind_select)
#     adata_temp = adata_temp[ind_select, 0:3]
#     adata_temp.X[:,0] = (adata_temp.obs['sex'] == 'male')*3
#     adata_temp.X[:,1] = (adata_temp.obs['condition'])*3
    # DE using MAST 
    R_cmd = util.call_MAST_age()
    get_ipython().run_cell_magic(u'R', u'-i adata_temp -i covariate -o de_res', R_cmd)
    de_res.columns = ['gene', 'raw-p', 'coef', 'bh-p']
    de_res.index = de_res['gene']
    DE_result_MAST[analyte] = pd.DataFrame(index = gene_name_list)
    DE_result_MAST[analyte] = DE_result_MAST[analyte].join(de_res)
    # fc between yound and old
    X = adata_temp.X
    y = (adata_temp.obs['condition']>10)
    DE_result_MAST[analyte]['fc'] = X[y,:].mean(axis=0) - X[~y,:].mean(axis=0)
#     break

Tongue 0/11
Heart_and_Aorta 1/11
Lung 2/11
Spleen 3/11
Liver 4/11
Bladder 5/11
Limb_Muscle 6/11
Thymus 7/11
Kidney 8/11
Marrow 9/11
Mammary_Gland 10/11


### Save DE results

In [9]:
with open(output_folder+'/DE_tissue_droplet.pickle', 'wb') as handle:
    pickle.dump(DE_result_MAST, handle)
    pickle.dump(analysis_list, handle)
    pickle.dump(analysis_info, handle)